In [61]:
import numpy as np
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
# from transformers.pipelines import pipeline #中文需要
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
import pandas as pd

In [62]:

# 加载数据
# step1 加载文件
with open('ghi_f_processed.txt', 'r', encoding='utf-8') as file:
    docs = file.readlines()
print('条数: ', len(docs))
print('预览第一条: ', docs[0])


条数:  11728
预览第一条:  unprecedented wealth creation inequitythe unjust class countriespersists reviving relevance social lens understand instrument intervene issue theoretical aspect polysemous character social applied field public assumed explicitly explained intersectional approach understanding argues preexisting class raceethnicity genderbased socially differentiated impact pandemic shaped interconnectedly economic maldistribution cultural misrecognition political misrepresentation pursuing requires analysis strategy intervention integrate economic cultural political sphere redistribution recognition representation intersectional approach relevant compelling light pandemic article broadly class raceethnicity gender political economy public healthbut narrower focus maldistribution misrecognition misrepresentation shaping social injustice



In [63]:
# 创建vectorizer_model
vectorizer_model = None
# 导入词嵌入模型
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
# 导入加载好的词向量

embeddings = np.load('emb_ghi_f.npy') # 加载回来，看一下效果
print(type(embeddings), embeddings.shape)

<class 'numpy.ndarray'> (11728, 384)


In [64]:
#设置固定随机数

# 2. 创建UMAP降维模型
umap_model = UMAP(
    n_neighbors=15,
    n_components=5,
    min_dist=0.0,
    metric='cosine',
    random_state= 42 # ⚠️ 防止随机 https://maartengr.github.io/BERTopic/faq.html
)

In [65]:

# 3. 创建聚类模型
# 创建HDBSCAN聚类模型
# 如果要建设离群值，可以减小下面两个参数
# https://hdbscan.readthedocs.io/en/latest/faq.html
hdbscan_model = HDBSCAN(
  min_cluster_size=40,
  min_samples=20,
  metric='euclidean',
  
)

In [66]:
# 4. 创建分词模型
vectorizer_model = CountVectorizer()  # 因为我们已经分好词了，所以这里不需要传入分词函数了
# 创建模型，跑出来的结果可能和我的不一样
topic_model = BERTopic(
  embedding_model=embedding_model,
  min_topic_size=20,
)
### 存档，保持不允许运行
# 训练模型
topic_model.fit_transform(docs, embeddings=embeddings)

# 获取聚类结果
topic_info = topic_model.get_topic_info()
topic_info


,Topic,Count,Name,Representation,Representative_Docs
0,-1,4204,-1_social_study_policy_care,"[social, study, policy, care, country, develop...",[background common mental impose substantial c...
1,0,868,0_vaccine_pandemic_vaccination_immunization,"[vaccine, pandemic, vaccination, immunization,...",[pandemic people life worldwide strategy appli...
2,1,316,1_food_agriculture_nutrition_production,"[food, agriculture, nutrition, production, sys...",[public issue century food special environment...
3,2,292,2_prison_criminal_disorder_mental,"[prison, criminal, disorder, mental, substance...",[people imprisoned worldwide individual experi...
4,3,267,3_maternal_mortality_birth_newborn,"[maternal, mortality, birth, newborn, neonatal...",[background progress reproductive maternal new...
...,...,...,...,...,...
79,78,22,78_pandemic_crisis_ecological_animal,"[pandemic, crisis, ecological, animal, ecology...",[purpose article examines official public resp...
80,79,21,79_alcohol_drinking_alcoholattributable_drunke...,"[alcohol, drinking, alcoholattributable, drunk...",[alcohol misuse substantial social issue austr...
81,80,21,80_medicine_price_availability_drug,"[medicine, price, availability, drug, essentia...",[backgroundessential medicine drug satisfy pri...
82,81,20,81_promotion_action_translation_ikt,"[promotion, action, translation, ikt, power, c...",[backgroundthe persistence inequity wicked str...


In [ ]:
# 可视化主题词
topic_model.visualize_barchart()


In [72]:
# 初步降维效果
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='euclidean').fit_transform(embeddings)
topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings, hide_document_hover=True, hide_annotations=True)

In [68]:
# 垂直聚类，关系分布
hierarchical_topics = topic_model.hierarchical_topics(docs)
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

100%|██████████| 51/51 [00:03<00:00, 14.38it/s]


In [ ]:
### hide_annotations=True hide_document_hover=True

In [78]:

#保存最终版
topic_docs = topic_model.get_document_info(docs)
topic_docs.to_csv('ghi_clustering_ceshi.csv')